# Week 12 Homework
## Question 15.2 Part 1
***Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)***
In the code block below, I first started with basic project set up by loading the necessary libraries, loading the data, and getting a quick view of the data. Previously, I opened the execl document manually and saw that there were a few trailing rows which contained constraints for this problem, these rows should not be included in the dataframe which is why I indexed the data.

In [56]:
# Load libraries
from pulp import *
import pandas as pd

# Read data
food_data = pd.read_excel('diet.xls')

# Convert data to list
food_data = food_data[0:64].values.tolist()

# Store food names
food_names = [i[0] for i in food_data]

food_data[0]

[u'Frozen Broccoli',
 0.16,
 u'10 Oz Pkg',
 73.8,
 0.0,
 0.8,
 68.2,
 13.6,
 8.5,
 8.0,
 5867.4,
 160.2,
 159.0,
 2.3]

In the next step of the problem I am going to create dictionaries to store each component of each food including cost, calories, cholesterol etc. I am using dictionaries to make it easier to loop later on when I start the pulp section of this problem. I printed an example of one of the dictionaries as well to show what each food dictionary looks like.

In [37]:
cost = dict([(i[0], float(i[1])) for i in food_data])
calories = dict([(i[0], float(i[3])) for i in food_data])
cholesterol = dict([(i[0], float(i[4])) for i in food_data])
total_fat = dict([(i[0], float(i[5])) for i in food_data])
sodium = dict([(i[0], float(i[6])) for i in food_data])
carbs = dict([(i[0], float(i[7])) for i in food_data])
fiber = dict([(i[0], float(i[8])) for i in food_data])
protein = dict([(i[0], float(i[9])) for i in food_data])
vitamin_A = dict([(i[0], float(i[10])) for i in food_data])
vitamin_C = dict([(i[0], float(i[11])) for i in food_data])
calcium = dict([(i[0], float(i[12])) for i in food_data])
iron = dict([(i[0], float(i[13])) for i in food_data])

print(cost)

{u'Roasted Chicken': 0.84, u'Tomato Soup': 0.39, u'Popcorn,Air-Popped': 0.04, u'Pretzels': 0.12, u'Bagels': 0.16, u'Crm Mshrm Soup,W/Mlk': 0.65, u'Splt Pea&Hamsoup': 0.67, u'White Tuna in Water': 0.69, u"Cap'N Crunch": 0.31, u'Macaroni,Ckd': 0.17, u'Couscous': 0.39, u'Kiwifruit,Raw,Fresh': 0.49, u'3.3% Fat,Whole Milk': 0.16, u'Banana': 0.15, u'New E Clamchwd,W/Mlk': 0.99, u'Skim Milk': 0.13, u'Carrots,Raw': 0.07, u"Raisin Brn, Kellg'S": 0.34, u'Malt-O-Meal,Choc': 0.52, u'Spaghetti W/ Sauce': 0.78, u'Chicknoodl Soup': 0.39, u'Frozen Corn': 0.18, u'Oranges': 0.15, u'Hamburger W/Toppings': 0.83, u'Wheat Bread': 0.05, u'Vegetbeef Soup': 0.71, u'Apple,Raw,W/Skin': 0.24, u'Neweng Clamchwd': 0.75, u'Grapes': 0.32, u'Chocolate Chip Cookies': 0.03, u'Potatoes, Baked': 0.06, u'White Bread': 0.06, u'Frankfurter, Beef': 0.27, u'White Rice': 0.08, u'Kielbasa,Prk': 0.15, u'Tomato,Red,Ripe,Raw': 0.27, u'Special K': 0.38, u'Sardines in Oil': 0.45, u'Lettuce,Iceberg,Raw': 0.02, u'Peppers, Sweet, Raw': 

In this next code block I am beginning the implementation of Pulp. This problem is a minimization problem and we are attempting to find the lowest cost while staying within the given food constraints provided in the excel doc. LpMinimize is the parameter we pass to LpProblem to make this a cost minimization model. After executing the minimization problem, I declared initial variables for each food. The lowBound for each variable is 0 because we can't eat a negative amount of something.

In [38]:
# Create minimization LpProblem
cheapest_diet = LpProblem("Diet Cost Minimization", LpMinimize)

# Initial variable for how much food we eat
foodVars = LpVariable.dicts("Foods", food_names, lowBound = 0 )

# Initial variable for whether we eat a specific food or not
chosenVars = LpVariable.dicts("Chosen", food_names, lowBound = 0, upBound = 1, cat = "Binary")

The next step is to create the objective function which will give us the total cost of ingredients in a diet. The objective function is added to the variable cheapest_diet which was declared above with LpProblem. After adding the objective function to the minimization model, I added in every diet constraint provided in the excel doc, which will enable pulp to solve for the minimum cost within the diet constraints.

In [39]:
# Add objective function to mimimize cost
cheapest_diet += lpSum([cost[i]*foodVars[i] for i in food_names]), "Total Cost"

# Add diet constraints to cheapest_diet model
cheapest_diet += lpSum([calories[i]*foodVars[i] for i in food_names]) >= 1500, 'min Calories'
cheapest_diet += lpSum([calories[i]*foodVars[i] for i in food_names]) <= 2500, 'max Calories'

cheapest_diet += lpSum([cholesterol[i]*foodVars[i] for i in food_names]) >= 30, 'min Cholesterol'
cheapest_diet += lpSum([cholesterol[i]*foodVars[i] for i in food_names]) <= 240, 'max Cholesterol'

cheapest_diet += lpSum([total_fat[i]*foodVars[i] for i in food_names]) >= 20, 'min fat'
cheapest_diet += lpSum([total_fat[i]*foodVars[i] for i in food_names]) <= 70, 'max fat'

cheapest_diet += lpSum([sodium[i]*foodVars[i] for i in food_names]) >= 800, 'min sodium'
cheapest_diet += lpSum([sodium[i]*foodVars[i] for i in food_names]) <= 2000, 'max sodium'

cheapest_diet += lpSum([carbs[i]*foodVars[i] for i in food_names]) >= 130, 'min Carbs'
cheapest_diet += lpSum([carbs[i]*foodVars[i] for i in food_names]) <= 450, 'max Carbs'

cheapest_diet += lpSum([fiber[i]*foodVars[i] for i in food_names]) >= 125, 'min fiber'
cheapest_diet += lpSum([fiber[i]*foodVars[i] for i in food_names]) <= 250, 'max fiber'

cheapest_diet += lpSum([protein[i]*foodVars[i] for i in food_names]) >= 60, 'min protein'
cheapest_diet += lpSum([protein[i]*foodVars[i] for i in food_names]) <= 100, 'max protein'

cheapest_diet += lpSum([vitamin_A[i]*foodVars[i] for i in food_names]) >= 1000, 'min vitamin A'
cheapest_diet += lpSum([vitamin_A[i]*foodVars[i] for i in food_names]) <= 10000, 'max vitamin A'

cheapest_diet += lpSum([vitamin_C[i]*foodVars[i] for i in food_names]) >= 400, 'min vitamin C'
cheapest_diet += lpSum([vitamin_C[i]*foodVars[i] for i in food_names]) <= 5000, 'max vitamin C'

cheapest_diet += lpSum([calcium[i]*foodVars[i] for i in food_names]) >= 700, 'min calcium'
cheapest_diet += lpSum([calcium[i]*foodVars[i] for i in food_names]) <= 1500, 'max calcium'

cheapest_diet += lpSum([iron[i]*foodVars[i] for i in food_names]) >= 10, 'min iron'
cheapest_diet += lpSum([iron[i]*foodVars[i] for i in food_names]) <= 40, 'max iron'

Now that the minimization model is defined with the objective function and food constraints, the next step is to run pulp with the method solve(). I also looped through the resulting variables to output which foods are included when the cost is minimized within the constraints. You can see from the output below that the foods included in the cheapest diet are raw celery, frozen broccoli, iceber lettuce, oranges, poached eggs, and popcorn which matches what homework instructions mentioned.

In [41]:
# Run pulp
cheapest_diet.solve()
print("Status:", LpStatus[cheapest_diet.status])

# Loop to print foods in the cheapest diet
for i in cheapest_diet.variables():
    if i.varValue != 0.0: 
        print(i.name, "=", i.varValue)

# Output total cost after minimizing
print ("Total Cost of meal with given dietary constraints is $%.2f" % value(cheapest_diet.objective))

('Status:', 'Optimal')
('Foods_Celery,_Raw', '=', 52.64371)
('Foods_Frozen_Broccoli', '=', 0.25960653)
('Foods_Lettuce,Iceberg,Raw', '=', 63.988506)
('Foods_Oranges', '=', 2.2929389)
('Foods_Poached_Eggs', '=', 0.14184397)
('Foods_Popcorn,Air_Popped', '=', 13.869322)
Total Cost of meal with given dietary constraints is $4.34


## Question 15.2 Part 2
Please add to your model the following constraints (which might require adding more variables) and solve the new model:
* If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
* Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
* To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected.

In this problem I started with the first additional constraint in bullet 1. I am making sure a food has an upper bound that matches whether it has been chosen or not. I also added a lower bound to make sure that if a food is chosen, 0.1 of the food is eaten and otherwhise 0 of the food is eaten. I used the binary variable declared above chosenVars to accomplish this. I added these constraints to the cheapest diet minimization model from above.

In [42]:
for i in food_names:
     cheapest_diet += foodVars[i] <= 10000000 * chosenVars[i]
     cheapest_diet += foodVars[i] >= .1 * chosenVars[i]

In the code block below, I added the constraint mentioned in bullet 2 from the instructions above. The code makes sure that there can only be one of broccoli or celery in case people don't like those foods.

In [43]:
cheapest_diet += chosenVars['Frozen Broccoli'] + chosenVars['Celery, Raw'] <= 1

In this next code block I handled the constraints outlined in bullet 3 which requires that we have at least 3 kinds of protein in our day to day meals to get some protein variety. 

In [44]:
cheapest_diet += chosenVars['Tofu'] + chosenVars['Roasted Chicken'] + \
chosenVars['Poached Eggs'] + chosenVars['Scrambled Eggs'] + chosenVars['Bologna,Turkey'] + \
chosenVars['Frankfurter, Beef']+chosenVars['Ham,Sliced,Extralean'] + \
chosenVars['Kielbasa,Prk']+chosenVars['Hamburger W/Toppings'] + \
chosenVars['Hotdog, Plain']+chosenVars['Pork'] +chosenVars['Sardines in Oil'] + \
chosenVars['White Tuna in Water'] >= 3

Finally, I executed the minimization function with these additional constraints. and output the foods contained in the new meal. You can see that the meal is only slightly higher at $4.51 per meal, but considerably more appetizing with the additional constraints. One thing I noticed however was the combination of poached and scrambled eggs, so in a future iteration I would take a look at adding a constraint to limit how many eggs are contained from the protein list.

In [46]:
cheapest_diet.solve()
print("Status:", LpStatus[cheapest_diet.status])

# Print out foods included in diet if they are non-zero
for i in cheapest_diet.variables():
    if i.varValue != 0.0:
        print(i.name, "=", i.varValue)

print ("Total Cost of the meal with added constraints is $%.2f" % value(cheapest_diet.objective))

('Status:', 'Optimal')
('Chosen_Celery,_Raw', '=', 1.0)
('Chosen_Kielbasa,Prk', '=', 1.0)
('Chosen_Lettuce,Iceberg,Raw', '=', 1.0)
('Chosen_Oranges', '=', 1.0)
('Chosen_Peanut_Butter', '=', 1.0)
('Chosen_Poached_Eggs', '=', 1.0)
('Chosen_Popcorn,Air_Popped', '=', 1.0)
('Chosen_Scrambled_Eggs', '=', 1.0)
('Foods_Celery,_Raw', '=', 42.399358)
('Foods_Kielbasa,Prk', '=', 0.1)
('Foods_Lettuce,Iceberg,Raw', '=', 82.802586)
('Foods_Oranges', '=', 3.0771841)
('Foods_Peanut_Butter', '=', 1.9429716)
('Foods_Poached_Eggs', '=', 0.1)
('Foods_Popcorn,Air_Popped', '=', 13.223294)
('Foods_Scrambled_Eggs', '=', 0.1)
Total Cost of the meal with added constraints is $4.51
